# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"s

gpu_num = 2

os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

In [2]:
!echo $CUDA_VISIBLE_DEVICES

2


In [7]:
### IJB-C Dataset
# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset
num_classes = 85742 
dataset_len = 5822653
batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size

w_decay=5e-4
epochs = 100

save_steps = 1000
steps = 1
is_ccrop=False
binary_img=True

is_Adam = False   # True
projection_head = False  # True
dgx = True

head_type = 'ArcHead' # ''ArcHead', CosHead', 'SphereHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception, MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
# Backbones w/o pretrained weights:
#      MnasNetA1, MnasNetB1, MnasNetSmall 
backbone_type = 'EfficientNetB6' 

if head_type == 'SphereHead':
    base_lr = 0.01 
    margin = 1.35
    logist_scale = 30.0 
elif head_type == 'CosHead':
    base_lr = 0.01 
    margin=0.35
    logist_scale=64
elif head_type == 'ArcHead':
#     base_lr = 0.01 
    base_lr = 0.1 
    margin=0.5
    logist_scale=64
else:
    base_lr = 0.01 # initially 0.01
    
print(head_type)
print(backbone_type)
print("projection head:", projection_head)
print("Adam:", is_Adam)
print("epoch:", epochs)
print("batch size:", batch_size)

ArcHead
EfficientNetB6
projection head: False
Adam: False
epoch: 100
batch size: 128


In [8]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

# print("data: ", train_dataset)


strategy = tf.distribute.MirroredStrategy(devices=[f"/gpu:{gpu_num}"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             margin=margin, 
                             logist_scale=logist_scale,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True,
                             projection_head=projection_head)
    model.summary()

    learning_rate = tf.constant(base_lr)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                                 decay_steps=60000, 
                                                   decay_rate=0.1,staircase=True)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule)
    else:
#         optimizer = tf.keras.optimizers.SGD(
#             learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb6 (Functional)     (None, 4, 4, 2304)   40960136    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          18886144    efficientnetb6[0][0]             
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [ ]:
from pathlib import Path
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

callbacks = [mc_callback, tb_callback, early_stopping]
# callbacks = [mc_callback, tb_callback, early_stopping, lr_scheduler]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/100
 1000/36391 [..............................] - ETA: 4:48:32 - loss: 51.4485
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_1_l_50.62767028808594.ckpt
 2000/36391 [>.............................] - ETA: 4:40:39 - loss: 50.8081
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_1_l_49.81047821044922.ckpt
 3000/36391 [=>............................] - ETA: 4:32:54 - loss: 50.3990
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_1_l_49.382728576660156.ckpt
 4000/36391 [==>...........................] - ETA: 4:24:31 - loss: 50.0953
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_1_l_48.956504821777344.ckpt
 5000/36391 [===>.....................

36000/36391 [============================>.] - ETA: 3:11 - loss: 44.9182
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_1_l_41.817840576171875.ckpt
36391/36391 [==============================] - 18609s 510ms/step - loss: 44.8846 - val_loss: 38.3278
Epoch 2/100
  609/36391 [..............................] - ETA: 4:51:37 - loss: 37.9701
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_2_l_37.91238021850586.ckpt
 1609/36391 [>.............................] - ETA: 4:43:59 - loss: 37.9281
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_2_l_37.88343048095703.ckpt
 2609/36391 [=>............................] - ETA: 4:35:49 - loss: 37.9012
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_Eff

33609/36391 [==========================>...] - ETA: 22:29 - loss: 36.8071
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_2_l_35.259342193603516.ckpt
34609/36391 [===========================>..] - ETA: 14:24 - loss: 36.7611
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_2_l_35.17231750488281.ckpt
35609/36391 [============================>.] - ETA: 6:19 - loss: 36.7153
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_2_l_35.08815383911133.ckpt
36391/36391 [==============================] - 18342s 504ms/step - loss: 36.6796 - val_loss: 31.8825
Epoch 3/100
  218/36391 [..............................] - ETA: 4:49:16 - loss: 32.4716
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_Efficie

31218/36391 [========================>.....] - ETA: 41:41 - loss: 31.6613
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_3_l_31.33820343017578.ckpt
32218/36391 [=========================>....] - ETA: 33:38 - loss: 31.6511
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_3_l_31.325143814086914.ckpt
33218/36391 [==========================>...] - ETA: 25:34 - loss: 31.6411
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_3_l_31.311410903930664.ckpt
34218/36391 [===========================>..] - ETA: 17:30 - loss: 31.6312
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_3_l_31.29723358154297.ckpt
35218/36391 [============================>.] - ETA: 9:27 -

28827/36391 [======================>.......] - ETA: 1:00:19 - loss: 30.2952
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_4_l_29.989784240722656.ckpt
29827/36391 [=======================>......] - ETA: 52:20 - loss: 30.2848
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_4_l_29.981714248657227.ckpt
30827/36391 [========================>.....] - ETA: 44:22 - loss: 30.2748
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_4_l_29.973384857177734.ckpt
31827/36391 [=========================>....] - ETA: 36:24 - loss: 30.2652
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_4_l_29.9647274017334.ckpt
32827/36391 [==========================>...] - ETA: 28:2

...

14481/36391 [==========>...................] - ETA: 2:56:42 - loss: 28.9269
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_10_l_28.892499923706055.ckpt
15481/36391 [===========>..................] - ETA: 2:48:36 - loss: 28.9248
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_10_l_28.894018173217773.ckpt
16481/36391 [============>.................] - ETA: 2:40:38 - loss: 28.9229
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_10_l_28.893421173095703.ckpt
17481/36391 [=============>................] - ETA: 2:32:35 - loss: 28.9211
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_10_l_28.890853881835938.ckpt
18481/36391 [==============>...............]

12090/36391 [========>.....................] - ETA: 3:19:24 - loss: 28.9327
Epoch 00011: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_11_l_28.878530502319336.ckpt
13090/36391 [=========>....................] - ETA: 3:10:52 - loss: 28.9288
Epoch 00011: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_11_l_28.88526725769043.ckpt
14090/36391 [==========>...................] - ETA: 3:02:19 - loss: 28.9259
Epoch 00011: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_11_l_28.889951705932617.ckpt
15090/36391 [===========>..................] - ETA: 2:53:54 - loss: 28.9236
Epoch 00011: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_11_l_28.8914794921875.ckpt
16090/36391 [============>.................] - 

 9699/36391 [======>.......................] - ETA: 3:37:18 - loss: 28.9451
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_12_l_28.871065139770508.ckpt
10699/36391 [=======>......................] - ETA: 3:29:12 - loss: 28.9378
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_12_l_28.86408805847168.ckpt
11699/36391 [========>.....................] - ETA: 3:21:07 - loss: 28.9318
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_12_l_28.873023986816406.ckpt
12699/36391 [=========>....................] - ETA: 3:13:01 - loss: 28.9276
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_12_l_28.882118225097656.ckpt
13699/36391 [==========>...................] 

 7308/36391 [=====>........................] - ETA: 3:55:16 - loss: 28.9676
Epoch 00013: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_13_l_28.88079833984375.ckpt
 8308/36391 [=====>........................] - ETA: 3:47:06 - loss: 28.9570
Epoch 00013: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_13_l_28.878446578979492.ckpt
 9308/36391 [======>.......................] - ETA: 3:39:07 - loss: 28.9483
Epoch 00013: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_13_l_28.87400245666504.ckpt
10308/36391 [=======>......................] - ETA: 3:30:59 - loss: 28.9407
Epoch 00013: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_13_l_28.864974975585938.ckpt
11308/36391 [========>.....................] -

 4917/36391 [===>..........................] - ETA: 4:11:15 - loss: 29.0149
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_14_l_28.880563735961914.ckpt
 5917/36391 [===>..........................] - ETA: 4:03:34 - loss: 28.9927
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_14_l_28.88624382019043.ckpt
 6917/36391 [====>.........................] - ETA: 3:55:47 - loss: 28.9773
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_14_l_28.885101318359375.ckpt
 7917/36391 [=====>........................] - ETA: 3:47:50 - loss: 28.9655
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_ArcHead_check/SGD/e_14_l_28.88096046447754.ckpt
 8917/36391 [======>.......................] -

### Resume training with latest checkpoint

In [6]:
from glob import glob
from pathlib import Path

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
file_list = []
for files in glob(f'{base_dir}{save_name}'):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(f'{base_dir}{save_name}'):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

13
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/e_14_l_12.577579498291016.ckpt


In [7]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{gpu_num}"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True,
                             projection_head=projection_head)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-lite0 (Functional) (None, 4, 4, 1280)   3413024     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    efficientnet-lite0[0][0]         
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [8]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 14/20
 1000/36391 [..............................] - ETA: 1:21:49 - loss: 12.4770
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_14_l_12.481388092041016.ckpt
 2000/36391 [>.............................] - ETA: 1:19:12 - loss: 12.4827
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_14_l_12.47399616241455.ckpt
 3000/36391 [=>............................] - ETA: 1:16:50 - loss: 12.4783
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_14_l_12.457012176513672.ckpt
 4000/36391 [==>...........................] - ETA: 1:14:34 - loss: 12.4737
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/che

24609/36391 [===================>..........] - ETA: 27:27 - loss: 12.2334
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_15_l_12.196710586547852.ckpt
25609/36391 [====================>.........] - ETA: 25:07 - loss: 12.2318
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_15_l_12.192112922668457.ckpt
26609/36391 [====================>.........] - ETA: 22:47 - loss: 12.2303
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_15_l_12.188915252685547.ckpt
27609/36391 [=====================>........] - ETA: 20:27 - loss: 12.2288
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidenti

18218/36391 [==============>...............] - ETA: 41:57 - loss: 12.0648
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_16_l_12.034358024597168.ckpt
19218/36391 [==============>...............] - ETA: 39:39 - loss: 12.0630
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_16_l_12.03027057647705.ckpt
20218/36391 [===============>..............] - ETA: 37:21 - loss: 12.0614
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_16_l_12.028227806091309.ckpt
21218/36391 [================>.............] - ETA: 35:03 - loss: 12.0598
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentit

11827/36391 [========>.....................] - ETA: 57:22 - loss: 11.9167
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_17_l_11.8868989944458.ckpt
12827/36391 [=========>....................] - ETA: 54:59 - loss: 11.9143
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_17_l_11.88347053527832.ckpt
13827/36391 [==========>...................] - ETA: 53:02 - loss: 11.9120
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_17_l_11.88149356842041.ckpt
14827/36391 [===========>..................] - ETA: 50:55 - loss: 11.9098
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/m

 5436/36391 [===>..........................] - ETA: 1:06:53 - loss: 11.7864
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_18_l_11.768781661987305.ckpt
 6436/36391 [====>.........................] - ETA: 1:04:42 - loss: 11.7834
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_18_l_11.765323638916016.ckpt
 7436/36391 [=====>........................] - ETA: 1:02:31 - loss: 11.7801
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_18_l_11.754659652709961.ckpt
 8436/36391 [=====>........................] - ETA: 1:00:22 - loss: 11.7772
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_

36391/36391 [==============================] - 4912s 135ms/step - loss: 11.7313 - val_loss: 11.8496
Epoch 19/20
   45/36391 [..............................] - ETA: 1:22:32 - loss: 11.9152
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_19_l_11.747794151306152.ckpt
 1045/36391 [..............................] - ETA: 1:18:44 - loss: 11.6741
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_19_l_11.657929420471191.ckpt
 2045/36391 [>.............................] - ETA: 1:16:36 - loss: 11.6689
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_19_l_11.644964218139648.ckpt
 3045/36391 [=>............................] - ETA: 1:14:15 - loss: 11.6

KeyboardInterrupt: 